In [ ]:
#!conda install pandas -y
#!pip install pandas

# **Chapter 2**
## *Reading Time Series Data from Files*, 

This chapter is an introduction to time series data. This chapter shows you how to read data from various and commonly used file
types, whether stored locally or on the cloud. The recipes will highlight advanced options
for ingesting, preparing, and transforming data into a time series DataFrame for
later analysis.  

We will cover the following recipes on how to ingest data into a pandas DataFrame:
* Reading data from CSVs and other delimited files
* Reading data from an Excel file
* Reading data from URLs
* Reading data from a SAS dataset

# Recipe 1: Reading from CSVs and Other Delimited Files

In [41]:
import pandas as pd
from pathlib import Path
pd.__version__

'2.0.2'

In [2]:
filepath = Path('../../datasets/Ch2/movieboxoffice.csv')

In [3]:
ts = pd.read_csv(filepath,
                 header=0,
             
                 parse_dates=[0],
                 index_col=0,
                 usecols=['Date',
                          'DOW',
                          'Daily',
                          'Forecast',
                          'Percent Diff'])
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


In [4]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DOW           128 non-null    object
 1   Daily         128 non-null    object
 2   Forecast      128 non-null    object
 3   Percent Diff  128 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


**Data Cleanup**

In [5]:
clean = lambda x: x.str.replace('[^\d]', '', regex=True)                                                      
c_df = ts[['Daily', 'Forecast']].apply(clean, axis=1)
ts[['Daily', 'Forecast']] = c_df.astype(float)
                                

In [6]:
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,12578989.0,23503646.0,-46.48%
2021-04-27,Saturday,9937401.0,19762255.0,-49.72%
2021-04-28,Sunday,8220316.0,11699126.0,-29.74%
2021-04-29,Monday,3353026.0,6665265.0,-49.69%
2021-04-30,Tuesday,3010524.0,3482819.0,-13.56%


In [7]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DOW           128 non-null    object 
 1   Daily         128 non-null    float64
 2   Forecast      128 non-null    float64
 3   Percent Diff  128 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.0+ KB


In [8]:
ts.memory_usage()

Index           1024
DOW             1024
Daily           1024
Forecast        1024
Percent Diff    1024
dtype: int64

In [9]:
ts.memory_usage().sum()

5120

## Using `date_parser`

In [10]:
ts.dtypes

DOW              object
Daily           float64
Forecast        float64
Percent Diff     object
dtype: object

In [11]:
date_parser = lambda x: pd.to_datetime(x, format="%d-%b-%y")
ts = pd.read_csv(filepath,
                 parse_dates=[0],
                 index_col=0,
                 date_parser=date_parser,
                 usecols=[0,1,3, 7, 6])
ts.head()

/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_12894/4043552452.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ts = pd.read_csv(filepath,


,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


In [12]:
ts = pd.read_csv(filepath,
                 header=0,
                 parse_dates=['Date'],
                 index_col=0,
                 infer_datetime_format= True,
                 usecols=['Date',
                          'DOW',
                          'Daily',
                          'Forecast',
                          'Percent Diff'])
ts.head()

/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_12894/2787762823.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  ts = pd.read_csv(filepath,
/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_12894/2787762823.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.read_csv(filepath,


,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


# Recipe 2: Reading data from an Excel file
* Learn how to read data from Excel files using pandas 
* Learn how to read from multiple sheets into DataFrames

* To install openpyxl using conda, run the following command in the terminal:

```
conda install openpyxl
```
* To install using pip, run the following command:

```
pip install openpyxl
```

In [13]:
#!conda install openpyxl -y
#!pip install openpyxl

In [2]:
import pandas as pd
from pathlib import Path
filepath = Path('../../datasets/Ch2/sales_trx_data.xlsx')

In [3]:
import openpyxl
openpyxl.__version__

'3.1.2'

In [16]:
excelfile = pd.ExcelFile(filepath)
excelfile.sheet_names

['2017', '2018']

In [17]:
excelfile.parse('2017')

,Line_Item_ID,Date,Credit_Card_Number,Quantity,Menu_Item
0,1,2017-01-01,7437926611570790,1,spicy miso ramen
1,2,2017-01-01,7437926611570790,1,spicy miso ramen
2,3,2017-01-01,8421920068932810,3,tori paitan ramen
3,4,2017-01-01,8421920068932810,3,tori paitan ramen
4,5,2017-01-01,4787310681569640,1,truffle butter ramen
...,...,...,...,...,...
36759,36760,2017-12-31,3149176814183170,4,vegetarian curry + king trumpet mushroom ramen
36760,36761,2017-12-31,2005206728506200,1,tonkotsu ramen
36761,36762,2017-12-31,2005206728506200,1,tonkotsu ramen
36762,36763,2017-12-31,1130414909788170,2,soft-shell miso crab ramen


In [18]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=[0,1],
                    parse_dates=True)
ts.keys()

dict_keys([0, 1])

In [19]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=['2017','2018'],
                    parse_dates=True)
ts.keys()

dict_keys(['2017', '2018'])

In [20]:
ts = pd.read_excel(filepath,
                    engine='openpyxl',
                    index_col=1,
                    sheet_name=None,
                    parse_dates=True)
ts.keys()

dict_keys(['2017', '2018'])

In [21]:
ts['2017'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36764 entries, 2017-01-01 to 2017-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        36764 non-null  int64 
 1   Credit_Card_Number  36764 non-null  int64 
 2   Quantity            36764 non-null  int64 
 3   Menu_Item           36764 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.4+ MB


In [22]:
ts['2018'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37360 entries, 2018-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        37360 non-null  int64 
 1   Credit_Card_Number  37360 non-null  int64 
 2   Quantity            37360 non-null  int64 
 3   Menu_Item           37360 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.4+ MB


In [23]:
ts_combined = pd.concat([ts['2017'],ts['2018']])

In [24]:
ts_combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 74124 entries, 2017-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        74124 non-null  int64 
 1   Credit_Card_Number  74124 non-null  int64 
 2   Quantity            74124 non-null  int64 
 3   Menu_Item           74124 non-null  object
dtypes: int64(3), object(1)
memory usage: 2.8+ MB


In [25]:
pd.concat(ts).index

MultiIndex([('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ('2017', '2017-01-01'),
            ...
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31'),
            ('2018', '2018-12-31')],
           names=[None, 'Date'], length=74124)

In [26]:
ts_combined = pd.concat(ts).droplevel(level=0)
ts_combined.head()

,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


In [27]:
ts = pd.read_excel(filepath,
                   index_col=1,
                   sheet_name='2018',
                   parse_dates=True)
type(ts)

pandas.core.frame.DataFrame

## There is more
* Using `ExcelFile`

In [28]:
excelfile = pd.ExcelFile(filepath)
excelfile.parse(sheet_name='2017',
                index_col=1,
                parse_dates=True).head()


,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


# Recipe 3: Reading data from a URL
* Learn how to read data as a DataFrame from GitHub
* Learn how to read data as a DataFrame from AWS S3 using `pandas`
* Learn how to read an HTML page and extract tables using `pandas`

In [29]:
#!conda install boto3 s3fs html5lib lxml -y
#!pip install boto3 s3fs html5lib lxml

* To install using pip, you can use the following command:

```
pip install boto3 s3fs lxml
```

* To install using Conda, you can use:

```
conda install boto3 s3fs lxml -y
```

In [4]:
import pandas as pd

In [5]:
import pandas as pd
import boto3, s3fs, lxml
print(f'''
pandas -> {pd.__version__}
boto3 -> {boto3.__version__}
s3fs -> {s3fs.__version__}
lxml -> {lxml.__version__}
''')


pandas -> 2.0.2
boto3 -> 1.26.54
s3fs -> 2023.5.0
lxml -> 4.9.2



## Reading from GitHub

In [6]:
# example of produced error
url = 'https://github.com/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook./blob/main/datasets/Ch2/AirQualityUCI.csv'
# pd.read_csv(url)


In [7]:
url = 'https://raw.githubusercontent.com/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook/main/datasets/Ch2/AirQualityUCI.csv'
date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%Y")

df = pd.read_csv(url,
                 delimiter=';',
                 index_col='Date',
                 date_parser=date_parser)

df.iloc[:3,1:4]

/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_13424/760453157.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(url,


,CO(GT),PT08.S1(CO),NMHC(GT)
Date,,,
2004-03-10,2.6,1360.00,150
2004-03-10,2.0,1292.25,112
2004-03-10,2.2,1402.00,88


## Reading from Public S3 Bucket

**Path-style access**

In [8]:
url = 'https://s3.us-east-1.amazonaws.com/tscookbook/AirQualityUCI.xlsx'

df = pd.read_excel(url,
                   index_col='Date',
                   parse_dates=True)
df.head()

,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date,,,,,,,,,,,,,,
2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


**Virtual-hosted–style access**

In [9]:
url = 'https://tscookbook.s3.amazonaws.com/AirQualityUCI.xlsx'
df = pd.read_excel(url,
                   index_col='Date',
                   parse_dates=True)
df.head()

,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date,,,,,,,,,,,,,,
2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


**Accessing a bucket using S3://**

In [10]:
s3uri = 's3://tscookbook/AirQualityUCI.xlsx'
df = pd.read_excel(s3uri,
                   index_col='Date',
                   parse_dates=True)
df.head()

,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date,,,,,,,,,,,,,,
2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


## Reading from Private S3 Bucket

In [20]:
import configparser
config = configparser.ConfigParser()
config.read('aws.cfg')

AWS_ACCESS_KEY = config['AWS']['aws_access_key']
AWS_SECRET_KEY = config['AWS']['aws_secret_key']


In [21]:
s3uri = "s3://tscookbook-private/AirQuality.csv"

df = pd.read_csv(s3uri,
                 index_col='Date',
                 parse_dates=True,
                 storage_options= {
                         'key': AWS_ACCESS_KEY,
                         'secret': AWS_SECRET_KEY
                     })

In [22]:
df.iloc[:3, 1:4]

,CO(GT),PT08.S1(CO),NMHC(GT)
Date,,,
2004-10-03,"2,6",1360.0,150.0
2004-10-03,2,1292.0,112.0
2004-10-03,"2,2",1402.0,88.0


**Using BOTO3**

In [28]:
import boto3
bucket = "tscookbook-private"
client = boto3.client("s3",
                  aws_access_key_id =AWS_ACCESS_KEY,
                  aws_secret_access_key = AWS_SECRET_KEY)


In [29]:
data = client.get_object(Bucket=bucket, Key='AirQuality.csv')

In [30]:
data.keys()

dict_keys(['ResponseMetadata', 'AcceptRanges', 'LastModified', 'ContentLength', 'ETag', 'ContentType', 'Metadata', 'Body'])

In [31]:
df = pd.read_csv(data['Body'],
                 index_col='Date',
                 parse_dates=True)


In [32]:
df.iloc[:3, 1:4]

,CO(GT),PT08.S1(CO),NMHC(GT)
Date,,,
2004-10-03,"2,6",1360.0,150.0
2004-10-03,2,1292.0,112.0
2004-10-03,"2,2",1402.0,88.0


## Reading from HTML

In [33]:
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
results = pd.read_html(url)
print(len(results))


69


In [34]:
# for i, k in enumerate(results):
#     print(i)
#     display(k.head())
    

In [35]:
df = results[15]
df.columns

Index(['Region[30]', 'Total cases', 'Total deaths', 'Cases per million',
       'Deaths per million', 'Current weekly cases', 'Current weekly deaths',
       'Population millions', 'Vaccinated %[31]'],
      dtype='object')

In [36]:
df[['Total cases', 'Total deaths', 'Cases per million']].head()

,Total cases,Total deaths,Cases per million
0,179537758,1185108,401363
1,103783777,1133607,281404
2,57721948,498259,247054
3,65835789,1313061,153151
4,25646533,434988,108307


## Example how `read_html()` works

In [37]:
import pandas as pd
html = """
 <table>
   <tr>
     <th>Ticker</th>
     <th>Price</th>
   </tr>
   <tr>
     <td>MSFT</td>
     <td>230</td>
   </tr>
   <tr>
     <td>APPL</td>
     <td>300</td>
   </tr>
     <tr>
     <td>MSTR</td>
     <td>120</td>
   </tr>
 </table>

 </body>
 </html>
 """

df = pd.read_html(html)
df[0]


,Ticker,Price
0,MSFT,230
1,APPL,300
2,MSTR,120


## Using `attr` option in `read.html()`

In [38]:
#!conda install html5lib beautifulSoup4

In [39]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
df = pd.read_html(url, attrs={'class': 'sortable'})
len(df)

5

In [40]:
df[3].columns

Index(['Region[30]', 'Total cases', 'Total deaths', 'Cases per million',
       'Deaths per million', 'Current weekly cases', 'Current weekly deaths',
       'Population millions', 'Vaccinated %[31]'],
      dtype='object')

# Recipe 4: Reading from a SAS dataset
* Dataset 1: http://support.sas.com/kb/61/960.html
* Learn about chunkisize (chunking) in pandas when working when reading large data files
* Learn about `Modin` using a separate Jupyter Notebook on how to improve performance with minimal effort 


In [49]:
import pandas as pd
path = '../../datasets/Ch2/DCSKINPRODUCT.sas7bdat'


In [50]:
df = pd.read_sas(path, chunksize=10000)
type(df)


pandas.io.sas.sas7bdat.SAS7BDATReader

In [51]:
results = []
for chunk in df:
    results.append(
        chunk)
len(results)

16

In [52]:
df = pd.concat(results)
df.shape

(152130, 5)

In [53]:
df = pd.read_sas(path, chunksize=10000)
results = []
for chunk in df:
    results.append(
        chunk.groupby('DATE')['Revenue']
             .agg(['sum', 'count']))
len(results)

16

In [54]:
pd.concat(results).shape

(1760, 2)

In [55]:
results[0].loc['2013-02-10']

sum      923903.0
count        91.0
Name: 2013-02-10 00:00:00, dtype: float64

In [56]:
results[1].loc['2013-02-10']

sum      8186392.0
count         91.0
Name: 2013-02-10 00:00:00, dtype: float64

In [57]:
results[2].loc['2013-02-10']

sum      5881396.0
count         91.0
Name: 2013-02-10 00:00:00, dtype: float64

In [58]:
from functools import reduce
final = reduce(lambda x1, x2: x1.add(x2, fill_value=0), results)
type(final)

pandas.core.frame.DataFrame

In [59]:
final.loc['2013-02-10']

sum      43104420.0
count        1383.0
Name: 2013-02-10 00:00:00, dtype: float64

In [60]:
final.shape

(110, 2)

## There is more

In [ ]:
# !conda install dask # install everything
#!conda install dask-core # install only core parts of dash
#!python -m pip install "dask[complete]"    # Install everything
#!python -m pip install dask                # Install only core parts of dask

# Recipe 4 - Read data from Parquet files

The following code reads the original data from NCEI NOAA in CSV then stores the data as Parquet files partitioned by year. The recipe focuses on reading the Parquet files and demonstrating how you can filter/query a specific partition.

In [35]:
import pandas as pd
from pathlib import Path
pd.__version__

'2.0.2'

In [2]:
file = 'https://www.ncei.noaa.gov/orders/cdo/3352259.csv'
df = pd.read_csv(file)
df['DT'] = pd.to_datetime(df['DATE'])
df['year'] = df['DT'].dt.year
df.to_parquet('../../datasets/Ch2/LA_weather.parquet', engine='pyarrow', partition_cols=['year'], compression='snappy')

Reading the Parquet files

In [39]:
file = Path('../../datasets/Ch2/LA_weather.parquet/')
df = pd.read_parquet(file,
                    engine='pyarrow')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4899 entries, 0 to 4898
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          4899 non-null   object        
 1   NAME             4899 non-null   object        
 2   DATE             4899 non-null   object        
 3   PRCP             4899 non-null   float64       
 4   PRCP_ATTRIBUTES  4899 non-null   object        
 5   SNOW             121 non-null    float64       
 6   SNOW_ATTRIBUTES  121 non-null    object        
 7   SNWD             59 non-null     float64       
 8   SNWD_ATTRIBUTES  59 non-null     object        
 9   TAVG             3713 non-null   float64       
 10  TAVG_ATTRIBUTES  3713 non-null   object        
 11  TMAX             4899 non-null   int64         
 12  TMAX_ATTRIBUTES  4899 non-null   object        
 13  TMIN             4899 non-null   int64         
 14  TMIN_ATTRIBUTES  4899 non-null   object 

Reading a specific partition for year 2010

In [7]:
file_2010 = Path('../../datasets/Ch2/LA_weather.parquet/year=2010/')
df_2010 = pd.read_parquet(file_2010)

In [10]:
df_2010.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          365 non-null    object        
 1   NAME             365 non-null    object        
 2   DATE             365 non-null    object        
 3   PRCP             365 non-null    float64       
 4   PRCP_ATTRIBUTES  365 non-null    object        
 5   SNOW             0 non-null      float64       
 6   SNOW_ATTRIBUTES  0 non-null      object        
 7   SNWD             0 non-null      float64       
 8   SNWD_ATTRIBUTES  0 non-null      object        
 9   TAVG             0 non-null      float64       
 10  TAVG_ATTRIBUTES  0 non-null      object        
 11  TMAX             365 non-null    int64         
 12  TMAX_ATTRIBUTES  365 non-null    object        
 13  TMIN             365 non-null    int64         
 14  TMIN_ATTRIBUTES  365 non-null    object   

Reading a specific partition for year 2012

In [50]:
filter = [('year', '==', 2012)]
df_2012 = pd.read_parquet(file, 
                          engine='pyarrow', 
                          filters=[filter])

Filtering to read files for years > 2020

In [51]:
df_2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          366 non-null    object        
 1   NAME             366 non-null    object        
 2   DATE             366 non-null    object        
 3   PRCP             366 non-null    float64       
 4   PRCP_ATTRIBUTES  366 non-null    object        
 5   SNOW             0 non-null      float64       
 6   SNOW_ATTRIBUTES  0 non-null      object        
 7   SNWD             0 non-null      float64       
 8   SNWD_ATTRIBUTES  0 non-null      object        
 9   TAVG             0 non-null      float64       
 10  TAVG_ATTRIBUTES  0 non-null      object        
 11  TMAX             366 non-null    int64         
 12  TMAX_ATTRIBUTES  366 non-null    object        
 13  TMIN             366 non-null    int64         
 14  TMIN_ATTRIBUTES  366 non-null    object   

In [12]:
df_2020 = pd.read_parquet(file, engine='pyarrow', filters=[('year', '>', 2020)])

In [44]:
filter = [('year', '>', 2020)]

In [45]:
pd.read_parquet(file, engine='pyarrow', filters=filter).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          881 non-null    object        
 1   NAME             881 non-null    object        
 2   DATE             881 non-null    object        
 3   PRCP             881 non-null    float64       
 4   PRCP_ATTRIBUTES  881 non-null    object        
 5   SNOW             0 non-null      float64       
 6   SNOW_ATTRIBUTES  0 non-null      object        
 7   SNWD             0 non-null      float64       
 8   SNWD_ATTRIBUTES  0 non-null      object        
 9   TAVG             881 non-null    float64       
 10  TAVG_ATTRIBUTES  881 non-null    object        
 11  TMAX             881 non-null    int64         
 12  TMAX_ATTRIBUTES  881 non-null    object        
 13  TMIN             881 non-null    int64         
 14  TMIN_ATTRIBUTES  881 non-null    object   

In [46]:
filter = [('year', '>=', 2021)]

In [48]:
pd.read_parquet(file, engine='pyarrow', filters= filter).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          881 non-null    object        
 1   NAME             881 non-null    object        
 2   DATE             881 non-null    object        
 3   PRCP             881 non-null    float64       
 4   PRCP_ATTRIBUTES  881 non-null    object        
 5   SNOW             0 non-null      float64       
 6   SNOW_ATTRIBUTES  0 non-null      object        
 7   SNWD             0 non-null      float64       
 8   SNWD_ATTRIBUTES  0 non-null      object        
 9   TAVG             881 non-null    float64       
 10  TAVG_ATTRIBUTES  881 non-null    object        
 11  TMAX             881 non-null    int64         
 12  TMAX_ATTRIBUTES  881 non-null    object        
 13  TMIN             881 non-null    int64         
 14  TMIN_ATTRIBUTES  881 non-null    object   

In [49]:
filter = [('year', 'in', [2021, 2022, 2023])]
pd.read_parquet(file, engine='pyarrow', filters=filter).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          881 non-null    object        
 1   NAME             881 non-null    object        
 2   DATE             881 non-null    object        
 3   PRCP             881 non-null    float64       
 4   PRCP_ATTRIBUTES  881 non-null    object        
 5   SNOW             0 non-null      float64       
 6   SNOW_ATTRIBUTES  0 non-null      object        
 7   SNWD             0 non-null      float64       
 8   SNWD_ATTRIBUTES  0 non-null      object        
 9   TAVG             881 non-null    float64       
 10  TAVG_ATTRIBUTES  881 non-null    object        
 11  TMAX             881 non-null    int64         
 12  TMAX_ATTRIBUTES  881 non-null    object        
 13  TMIN             881 non-null    int64         
 14  TMIN_ATTRIBUTES  881 non-null    object   

In [14]:
df_2020.head()

,STATION,NAME,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,SNWD,SNWD_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES,DT,year
0,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2021-01-01,0.0,",,W,2400",NaN,None,NaN,None,57.0,"H,,S",67,",,W",43,",,W",2021-01-01,2021
1,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2021-01-02,0.0,",,W,2400",NaN,None,NaN,None,54.0,"H,,S",63,",,W",46,",,W",2021-01-02,2021
2,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2021-01-03,0.0,",,W,2400",NaN,None,NaN,None,54.0,"H,,S",62,",,W",49,",,W",2021-01-03,2021
3,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2021-01-04,0.0,",,W,2400",NaN,None,NaN,None,54.0,"H,,S",59,",,W",48,",,W",2021-01-04,2021
4,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2021-01-05,0.0,",,W,2400",NaN,None,NaN,None,54.0,"H,,S",57,",,W",50,",,W",2021-01-05,2021


In [13]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          881 non-null    object        
 1   NAME             881 non-null    object        
 2   DATE             881 non-null    object        
 3   PRCP             881 non-null    float64       
 4   PRCP_ATTRIBUTES  881 non-null    object        
 5   SNOW             0 non-null      float64       
 6   SNOW_ATTRIBUTES  0 non-null      object        
 7   SNWD             0 non-null      float64       
 8   SNWD_ATTRIBUTES  0 non-null      object        
 9   TAVG             881 non-null    float64       
 10  TAVG_ATTRIBUTES  881 non-null    object        
 11  TMAX             881 non-null    int64         
 12  TMAX_ATTRIBUTES  881 non-null    object        
 13  TMIN             881 non-null    int64         
 14  TMIN_ATTRIBUTES  881 non-null    object   

In [23]:
pd.read_parquet(file, engine='pyarrow', filters=[('year', '>', 2020)], columns=['DATE', 'year', 'TMAX'])

,DATE,year,TMAX
0,2021-01-01,2021,67
1,2021-01-02,2021,63
2,2021-01-03,2021,62
3,2021-01-04,2021,59
4,2021-01-05,2021,57
...,...,...,...
876,2023-05-27,2023,67
877,2023-05-28,2023,64
878,2023-05-29,2023,68
879,2023-05-30,2023,66


In [25]:
pd.read_parquet(file, engine='pyarrow', filters=[('year', 'in', [2010, 2023])], columns=['DATE', 'year', 'TMAX'])

,DATE,year,TMAX
0,2010-01-01,2010,69
1,2010-01-02,2010,76
2,2010-01-03,2010,80
3,2010-01-04,2010,75
4,2010-01-05,2010,76
...,...,...,...
511,2023-05-27,2023,67
512,2023-05-28,2023,64
513,2023-05-29,2023,68
514,2023-05-30,2023,66
